In [5]:
!echo "Y" | pip uninstall compact

Found existing installation: compact 0.0.1
Uninstalling compact-0.0.1:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/compact-0.0.1.dist-info/*
    /usr/local/lib/python3.11/dist-packages/compact/*
Proceed (Y/n)?   Successfully uninstalled compact-0.0.1


# Starter Kit: Contextual Performance Profiling and Anomaly Detection

In [9]:
!pip install -r https://raw.githubusercontent.com/rpverbek/compact-sk/main/compact/requirements.txt
try:
    # running on colab
    from google.colab import output
    output.enable_custom_widget_manager()
except ModuleNotFoundError:
    # running locally
    display("Not running in Google Colab, skipping the Colab-specific code.")

  Cloning https://github.com/rpverbek/compact-sk.git to /tmp/pip-install-gpuvf24f/compact_f56ad6ac9e294c45ac348238776db999
  Running command git clone --filter=blob:none --quiet https://github.com/rpverbek/compact-sk.git /tmp/pip-install-gpuvf24f/compact_f56ad6ac9e294c45ac348238776db999
  Resolved https://github.com/rpverbek/compact-sk.git to commit 9649668756c46753c2b1eda3b5472525d2a4e957
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
ERROR: Could not open requirements file: [Errno 2] No such file or directory: '/compact/requirements.txt'

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


'Not running in Google Colab, skipping the Colab-specific code.'

In [10]:
%load_ext autoreload
%autoreload 2
# The autoreload can be removed at the end?
from compact.data.phm_data_handler import fetch_and_unzip_data
from compact.nmf_profiling import extract_nmf_incremental, get_df_W_offline_and_online
from compact.nmf_profiling import get_pivot_table
from compact.viz.viz import illustrate_nmf_components_interactive, show_fingerprints, plot_example_interactive
from compact.viz.viz import plot_ROC_curve, plot_weights_interactive
from compact.util import get_operating_modes
from compact.preprocessing import get_and_preprocess_healthy_data, get_and_preprocess_unhealthy_data
import seaborn as sns

from IPython.display import Markdown as md

# ignore convergence warnings (1000 iterations reached by NMF)
import warnings;
warnings.filterwarnings('ignore');

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload




> ToDo: make clear that Anomaly Detection and Performance Profiling has their own code section.


## Business context
Industry 4.0 leverages on the advanced AI technologies to enable anomaly detection and performance profiling of industrial assets operating in different contexts. Context is defined by both internal and external factors, such as operational conditions, environmental variables, and usage patterns. For this reason, context-aware methods are fundamental to identify anomalies and to ensure accurate and reliable asset profiling. These methods allow for real-time monitoring and enable enhanced performance and reduce downtime of assets.

## Business goal

The business goal related to this Starter Kit (SK) is to illustrate a data-driven methodology to identify anomalies and profile the performance of assets operating in different contexts, i.e. in terms of process measurements reflecting the internal operations of the asset.
As data-driven methodology, this SK focuses on the methodology developed by Fingerhut et al. [[1](#fingerhut2023), [2](fingerhut2024)]. Conventional anomaly detection methods often detect anomalies when the operating conditions change, rendering them less applicable for real-world dynamic industrial use cases. In contrast, the methodology presented in this starterkit is suitable for these scenarios, because it considers the dynamic nature of operating conditions.


## Application contexts

Contextual anomaly detection and performance profiling play a relevant role in a variety of industrial contexts such as:

- Raise warnings to anticipate and avoid safety-critical conditions

- Alert the need for inspection to avoid possible downtime and cost corrective maintenance

- Performance benchmarking

**FFNG: Do we use double spacing between some bullet points (like the one above) on purpose.**

## Data characteristics and requirements
To showcase the SK, a dataset is required that includes:

- The vibration frequency captured by asset sensors. The captured vibrations need to be related to the health state of the assets.
- Parameters related to the internal operations of the asset.

## Starter Kit outline


The SK is organized in four main sections. First, the required background knowledge is provided to understand the terminology used in the rest of the document. Second, a description of data generated generated by the assets along with its preprocessing is reported. Third, the methodology introduced by [Fingerhut et al. [1]](#fingerhut2023) is illustrated highlighting how it can be used for performance profiling and anomaly detection of assets. Finally, conclusions are drawn.

At the end of the SK you will know how to:

- Develop a model for anomaly detection and performance profiling

- Experimentally validate the resulting model and objectively compare it with other approaches


## 1. Background
  
- **anomaly**: A data point that deviates significantly from what is expected under the given conditions.
- **operating mode**. An asset can operate in different contexts which can influence its behavior. We refer to these context-dependent behaviors as operating modes. As an example we can imagine a gearbox that operates in two modes: normal and throttle. These two modes can be isolated by analzying the context, specifically the rotation speed of the gears. In throttle mode, the gearbox would operate under a reduced load, which would be indicated by a low rotational speed.
- **operating and performance views**. The operating view is composed of the parameters capturing the operating context (e.g. rotation speed, torque). The performance view is composed of the parameters monitoring the performance behaviour (e.g. vibrations).

## 2. Data understanding

### 2.1 Data description

The dataset we will use in this Starter Kit comes from the [PHM North America challenge '23](https://data.phmsociety.org/phm2023-conference-data-challenge/). This dataset collect the time series data from a gearbox subject to pitting, i.e. a fatigue failure of the gear tooth along with metadata. This dataset includes measurements under varied operating conditions from a healthy state as well as six known fault levels. The training data are collected from a range of different operating conditions under 15 different rotational speeds and 6 different torque levels.  For each operating condition, 5 vibration measurements were collected.
The vibration data is given in the time domain with a sampling rate of 20480Hz. The sampling duration differs between 3 seconds and 12 seconds. For each vibration measurement there are triaxial time-domain vibration measurements available (x, y and z). The vibrations are collected at different rotation per minute (rpm) and different runs.  Below, the user can get acquainted with the [[dataset](https://data.phmsociety.org/phm2023-conference-data-challenge/)] by visualizing the vibration measurements in the three directions for different rpm and runs. [**FFNG: In its current form, it might not be entirely clear how different vibration directions are handled. --> Need to go over that. AMUR: I did an iteration. Looks better? --> FFNG: Yes, thanks!**]



<img src="https://data.phmsociety.org/wp-content/uploads/sites/9/2023/06/PHM2023dc_fig1.png" alt="MarineGEO circle logo" style="height: 375px; width:800px;"/>

In [11]:
# Fetch the data if necessary
fetch_and_unzip_data()

Unzipping data...


BadZipFile: File is not a zip file

In [ ]:
plot_example_interactive()

### 2.2 Preprocessing

Splitting the dataset into training and testing sets is a common activity for validating the machine learning models. The training dataset is used to prepare a model, i.e. to train it, whereas the testing dataset is used to evaluate the performance of the model.
In our case, training and a testing set are based on splitting the vibration measurements. More specifically, a random sample of 75% of the original data recorded under normal (= healthy) condition is used for training whereas the remaining 25% is used for testing.  In the rest of the document, we will only refer to this split to analyze the performance of the methodology for the sake of the computational time. However, to have more reliable results, multiple splits should be performed, i.e. multiple random samples should be extracted for the training and testing sets. The interested reader can refer to [Fingerhut et al. [2]](#fingerhut2024) to see the results when 100 random splits are generated.



The test set is then created by combining:
1. **Normal condition**: The 25% healthy data that was held back (not used in the training set)
2. **Anomaly condition**: Vibration data characterized by pitting level 1-8 (two more than in the training set). For each level of pitting, there are between 267 and 304 samples in the test set that were recorded at different speeds and torques.

The figure below illustrates the train-test split.

<img src="https://github.com/rpverbek/compact-sk/blob/main/work/figures/overview_train-test-split.png?raw=1" alt="Overview train-test split" style="width:1000px;"/>

Furthermore, combinations of rotational speed X and torque Y are excluded from the test set, if they do not occur in the training set. This situation occurs, if the random train-test split did by chance assign all measurements with the same operating parameters to the test set.  [**FFNG: Added this sentence. (21.08.)**]. **AMUR: I don't get this sentece. I could be possible that they do not occur? Are there sample where X and Y are not recorded? If this is the case, should they not be removed even from the training set? --> FFNG: As a result of the random train-test split, some of the operating modes only occur in the test set (let's take for instance all measurements at 500 rpm and 100 Nm). If they cannot be found in the training set, then no vibration fingerprints would be constructed in the training set. I added this explanation in the sentence before.**

As the next step, the original data is prepared in the training set for the analysis.
- The **time series data** is transformed into frequency-bands. Typically, a frequency band expresses vibration behaviour in a specific range of frequencies (measured in Hertz [Hz]). We define a frequency-band in terms of "orders". It is worth to notice that a frequency measure captures the number of events per second whereas the "orders" capture the number of events per revolution of the rotating element. Finally, a order-transformation is applied to standardize the data. This step is important and a common preprocessing step in prognostics and health management. For details, the interested reader is referred to the vibration alignment section of [Fingerhut et al. [1]](#fingerhut2023). All order-transformed vibration measurements are organized in a matrix which we call **performance matrix V**.
- Metadata is created from the original dataset. The metadata contains the parameters at which the vibrations were measured:
    - The torque expresses the rotational force in terms of Newton meter.
    - The rotation speed expresses, how fast the gearbox is rotating in terms of revolutions per minute.


A brief excerpt of the **performance matrix V** is shown below.

In [ ]:
# Here we need something to showcase how the training and testing dataset looks like (columns, their meaning, etc)
df_V_train, meta_data_train, df_data_healthy_test, f = get_and_preprocess_healthy_data()
df_orders_test, meta_data_test = get_and_preprocess_unhealthy_data(df_data_healthy_test, f)

# extract list of frequency band columns for later usage
cols_ = df_V_train.columns
BAND_COLS = cols_[cols_.str.contains('band')].tolist()

df_V_train.head()

The matrix above has n columns which represent vibration measurements and m rows which represent the frequency bins.

Next, we illustrate the corresponding metadata, consisting of the previously described process parameters `rotational speed [RPM]` and `torque [Nm]`, and some additional information regarding the vibration measurement direction (`direction`) and sample ids (`sample_id` and `unique_sample_id`). Each row corresponds to the same row in the performance matrix. For instance, the very first measurement contains vibrations recorded at 100 RPM and 300 NM, for the vibration measurement direction x.

In [ ]:
meta_data_train.head()

The number of operating modes is given by the unique combinations of `rotational speed [RPM]` and `torque [Nm]`. There are 76 unique operating modes.

In [ ]:
n_oms = len(meta_data_train['unique_sample_id'].str[:-2].unique())
display(md(f'There are {n_oms} unique operating modes.'))

## 3. Methodology

The methodology can be divided into the offline and online phases. The offline phase focuses on performance profiling. More specifically, the operating modes of healthy assets are extracted and mapped to the expected performances. The online phase focuses on anomaly detection by exploiting the performance profiles extracted in the previous phase.


### 3.1 Offline phase

The general workflow of the offline phase is reported in the figure below.

<img src="https://github.com/rpverbek/compact-sk/blob/main/work/figures/overview_offline-phase.png?raw=1" alt="Overview offline phase" style="width:800px;"/>


This phase can be divided into 3 steps (See Figure above). The main goals of these phases can be summarized as follow:
1. Characterizing performance  behaviour. High-dimensional vibration signals are characterized in terms of a few vibration components that capture the fundamental vibration behaviour.
2. Extracting operating modes. Based on the context, the extracted operating modes are associated with each timestamp.

3. Linking operating modes to performance behaviour. For each operating mode a performance fingerprint is associated.



#### Characterizing performance behaviour

In this step, we extract for each asset its characteristic **performance behaviour**. These are extracted by applying a _non-negative matrix factorization_ (NMF) [[3](#nmf)] to the performance matrix V. The NMF method approximates a given matrix with two lower-dimensional matrices. The approximation contains the essential information about the asset's performance. Hence, NMF reveals underlying patterns in the performance data by representing it in a simpler form.

We apply NMF to decompose the matrix V into a separate weight matrix $\mathbf{W}$ and a component matrix $\mathbf{H}$, i.e., $\mathbf{V} \approx \mathbf{W} \times \mathbf{H}$.
Compared to other decomposition methods, NMF has the advantage that due to the entirely positive values, $\mathbf{H}$ is interpretable.

The component matrix $\mathbf{H}$ contains a set of $h$ representative components allowing to express performance behaviour in a standardized way and thus allowing to compare performance across operating modes and across assets. The matrix $\mathbf{W}$ contains the weights for reconstructing the original performance matrix $\mathbf{V}$. Each element in $\mathbf{W}$ can be interpreted as the weights of the building blocks in $\mathbf{H}$ needed to reconstruct a vibration signal encoded in the performance matrix $\mathbf{V}$.

To decide the value $h$ of components we decompose the matrix into an increasing number of components. In the next code section, we apply the NMF to up to 50 components. Unlike PCA, the decompositions cannot be derived from each other. Therefore, we need to recalculate them for every iteration.

In [ ]:
MAX_N_COMPONENTS = 50    # maximum number of components used to recompute

df_nmf_models = extract_nmf_incremental(df_V_train, max_n_components=MAX_N_COMPONENTS, timestamps=df_V_train.index, verbose=True)

Below, we illustrate the decomposition of the vibration data.

The identification of the number of components to describe the operating context is not trivial. To facilitate this process, in the plot below we provide the following plots.
- The top two plots visualize how well the performance matrix $\mathbf{V}$ can be approximated
    - The left-top plot illustrates the cumulative explained variance of a principal component analysis (PCA) of the performance matrix $\mathbf{V}$. It merely serves as an indication of an upper bound for how well the signal could be expressed using PCA.
    - The top-righ plot illustrates the reconstruction error of the performance matrix $\mathbf{V}$ using the NMF components. The reconstruction error is calculated as the Frobenius norm of the difference between the original matrix and the reconstructed matrix.
- The bottom lineplots illustrate the NMF components.

In the interactive widget below, the reader can select how the number of components are determined. This number can be based on:
- the reconstruction error plot. In this case, the _knee point_ is identified and used to determine the number of components. The identification of this knee point is an extension of the methodology presented in [2] for automated hyperparameter tuning for operative context detection.

- a threshold for the cumulative _explained variance_ from PCA (e.g., 95%). In this case, the smallest number of components for which this threshold is exceeded, is selected.
- both the aforementioned methods. In this case the highest of the two values is used as the number of components.

Each of the $h$ components from $\mathbf{H}$ is illustrated in a separate lineplot. The components serve as building blocks of the observed vibration signals, revealing common patterns in the vibration measurements.

For the suggested settings, it can be observed that components 1 and 2 form the basis vectors for peaks that are observed at 40 and 80 orders. These peaks are expected, as the driving gear has 40 teeth. It is common to see peaks for multiples of the number of teeth, hence there is also a peak at 80 orders. Components 3, 4 and 5 encode smaller peaks which model the noise floor.

The reader is invited to experiment with multiple parameters below to verify how they affect the number of components extracted. The domain expert can use this code section to incorporate their domain knowledge to make the extraction and representation of implicit contexts more data-efficient .

The interactive widget allows to go beyond the parameters selected in the original methodology [[2](#fingerhut2024)]. By increasing the range of acceptable parameters or by increasing the selection criterion (e.g., the explained variance), more components will be used in the decomposition. This makes the vibration fingerprints more granular, but also more prone to noise. In contrast, decreasing the accepted range or the selection criterium will yield less components, making the resulting fingerprints less informative, but also less prone to noise.

The number of components should be neither too low, which would lead to inaccurate, too general vibration fingerprints, nor should it be too high, which would lead to too specific, fingerprints prone to noise.

In [ ]:
saved_values = illustrate_nmf_components_interactive(df_V_train, df_nmf_models)

We proceed with the number of components determined in the interactive figure above.

In [ ]:
N_COMPONENTS = saved_values['n_components']
COMPONENT_COLUMNS = list(range(N_COMPONENTS))  # used later
model = df_nmf_models[(df_nmf_models.n_components == N_COMPONENTS)].iloc[0]
display(md(f'We are using **{N_COMPONENTS} components**.'))

> At the end of this section, the reader played with the number of components. Did we test multiple options? Should we not put a limit to avoid that the notebook breaks?

While the starterkit is able to cope with different hyperparameters, in the text that follows, we assume that at least 5 components were chosen.

#### Extracting operating modes.

> AMUR: In your opinion Fabian, is this the best place where I can write: here we perform "Context segmentation". I need to repor these two words somewhere. FFNG: Yes: "context" expressed in "operating modes"; "context segmentation" $\approx$ "extraction of operating modes". Let's write it in the title such that it is clear?

In this step, we extract the operating modes of the assets. In the use case, a finite number of speed and torque values are tested.
For clarity, in the text that follows, the operating modes are reported using the pattern `@ X rpm, Y Nm`, where `X` corresponds to the rotational speed and `Y` corresponds to the torque. All measurements with the same rotational speed and torque are summarized in the same operating mode (OM). There are as many operating modes, as there are unique combinations of `X` and `Y`.

In [ ]:
df_operating_modes = get_operating_modes()
df_operating_modes

The pivot table above depicts the names of the extracted operating modes in function of the two operating parameters speed and torque: An asset running in `OM 1` (top left) runs at a rotational speed of 100 rpm and a torque of 50 Nm, whereas an asset running in `OM 77` (bottom right) runs at 1200 rpm and 500 Nm. For this train-test split, there is a total of 77 operating modes.
> AMUR: Fabian can you continue? --> FFNG: Adapted description.

#### <a id='Link-operating-modes-to-performance-behaviour'>Linking operating modes to performance</a>
In this step, the performance behaviour is linked with the operating modes. This allows to derive **context-sensitive performance fingerprints**. As each vibration measurement is assigned to an operating mode, it is possible to derive fingerprints by aggregating all rows in $\mathbf{W}$ annotated with the same operating mode.
In the following, for each of the 77 operating modes, the reader can visualize the performance behaviour for each individual measurement based on the selected statistics (e.g. mean).




In [ ]:
show_fingerprints(model, df_V_train, meta_data_train, df_operating_modes)

> AMUR: Fabian can introduce in the text that follow psd? I don't know what it means. If you could mention it, it would help. Please complete the next sentence

The table reports in x-axis ..., y-axis, the color describes ...  [complete sentence].
From the analysis of the operating modes, it is possible to observe that they present distinct performance fingerprints, as can be observed for instance when comparing e.g., operating mode (OM) 1 with OM 50. Whereas OM 1 predominantly expresses vibrations in the third component related to the noise floor for all vibration directions, OM 50 predominantly expresses vibrations in the first two components related to 40 and 80 orders. At the same time, operating modes with similar operating conditions show similar vibration fingerprints. For instance, OM 1 and OM 16 only differ slightly in the torque, which results (as expected) in similar vibration fingerprints.

At the end of this offline phase, each operating mode has its own performance profile

### 3.2 Online phase

The general workflow of the online phase is reported in the figure below. Note that due to the nature of the data used in this use case, some steps are no longer necessary or become very simplified. These steps are still included for the sake of completeness.

<img src="https://github.com/rpverbek/compact-sk/blob/main/work/figures/overview_online-phase.png?raw=1" alt="Overview online phase" style="width:800px;"/>

> Part of the text below should be simplified [I need a lot of help from Fabian for this job]. --> FFNG: Is it simplified enough? 

This phase can be divided in five steps (See Figure above). The main goals of these phases can be summarized as follows:

4. **Windowing incoming streaming**. Streaming data is usually divided into batches. This activity is generally performed to avoid processing each new received data point as it arrives. However, since the data isn't streaming in this case, windowing isn't needed, so this step is skipped.

5. **Detecting of the operating context**. Each time stamp is associated with an operating mode which was identified during the offline phase. In this use case, operating modes are linked to vibration measurements without time stamps.

6. **Deriving the performance profiles**. Each vibration measurement is characterized in terms of the vibration components that were extracted in step 1 during the offline phase.

7. **Estimating the fingerprint offset**. For each vibration measurement, the _offset_ is calculated between the online profiles and the offline fingerprints. The offset quantifies to what extent the observed vibration behaviour differs from the expected vibration behaviour expressed through the fingerprints.

8. **Deriving alarms**. In the original paper [[2](#fingerhut2024)], based on the offset, an anomaly score is computed for each time stamp. This score is monitored over time raise alarms. In this use case, the monitoring of the anomaly score is not possible since the timestamps in the test set are not ordered. For this reason, this step is skipped.


#### Detecting of the operating context and deriving the performance profiles.

As in the offline phase, the operating modes are extracted from each timestamp present in the testing dataset. Then, the performance profiles are derived for each vibration measurement.

The goal of extracting the performance profiles is to capture and represent the key characteristics of the vibration signal in a way that reflects the asset's vibration behavior. Therefore, this step essentially mirrors the step 1 of the offline phase, with the exception that the vibration components are not derived again.

The performance profiles approximate the vibration signal in terms of the representative vibration components that were extracted during step 1 of the offline phase. Similarly to this step, the approximation contains the essential information about the asset's performance.
These profiles are extracted by applying NMF to the processed vibration measurements from the online phase, denoted as $\mathbf{v}'$.
The processed measurements $\mathbf{v}'$ are a vector with the same number of elements as the columns in the performance matrix $\mathbf{V}$. Since the step is applied to a data stream, multiple samples are no longer stacked together as in $\mathbf{V}$ during the offline phase.
> AMUR; Fabian is not clear the last sentence. What is denoted as $\mathbf{v}'$? Could rephrase the sentence? --> FFNG: I added some explanation. Is it now more clear?

In the online phase, NMF now uses the $h$ components from matrix $\mathbf{H}$, which were already identified during the offline phase. Specifically, NMF decomposes $\mathbf{v}'$ into a weight vector $\mathbf{w}'$ using the fixed component matrix $\mathbf{H}$, such that $\mathbf{v}' \approx \mathbf{w}' \times \mathbf{H}$.

Since the same components are used from the offline phase, the profiles match the format of the vibration fingerprints, making them suitable for subsequent anomaly detection steps.

In [ ]:
df_W_offline, df_W_online, fingerprints, test_vibration_measurement_periods_meta_data = get_df_W_offline_and_online(df_V_train, meta_data_train, meta_data_test, model, df_orders_test)

To better understand the derived weights $\mathbf{w}'$, below the derived weights for a single measurement are illustrated
A vibration measurement consists of the three measurements directions (namely the X-, Y- and Z-axis, corresponding to the rows in the colormap below).
The weights that are illustrated below consist of $h$ columns each, where $h$ represents the number of components determined in the offline phase.
The format of the weights is the same as that of the fingerprint shown in the [previous section](#Link-operating-modes-to-performance-behaviour).

In [ ]:
plot_weights_interactive(df_W_online, meta_data_test, df_operating_modes)

As an example, we select measurement 10 from the test set, which was taken at 1000 rpm and 50 Nm and therefore corresponds to operating mode 10. It can be observed that components 1 and 4 have high values in all 3 measurement directions.
> AMUR: Fabian what is a strong presence?. I don't think the reader even knows what is the meaning of code color (ie. the meseaurement range).
>
> RVEB: I changed it to "high values" instead of high presence.

When comparing this to the vibration fingerprint at OM 10 (see the [interactive widget for showing the fingerprints](#Link-operating-modes-to-performance-behaviour) and choose operating mode 10), we observe a similar behaviour. Hence, for this example, the observed beahviour is similar to the expected vibration behaviour, which is indicative that this vibration measurement can be considered as non-anomalous. This comparison is a representative example on how to check whether the vibration measurement is indicative of an anomaly.

At the end of this step, for each online vibration measurement, a performance profile is extracted, which is of the same format as the vibration fingerprints.
> AMUR: Fabian what does it mean: "processed to a corresponding performance". Can you be more clear? --> FFNG: I changed the sentence. Is it now more clear?

The performance profile contains the essential information of the vibration signal, while being of the same format as the vibration fingerprints.
> AMUR: Fabian do we need the sentence above? --> FFNG: I added the information in the previous sentence.

In the next section, the performance profiles are compared to the expected vibrations from the fingerprints in an automated fashion.

#### Estimating the fingerprint offset

In order to assess
whether the performance of asset $i$ ($i = 1, . . . , n$) is normal or anomalous, it is necessary to quantify the distance - offset - between the observed and expected fingerprints. For this reason, for each performance measurement, the cosine distance between the derived weights $\mathbf{w}'$ and the fingerprint $f_{ij} \in F_i$, corresponding to the detected operating mode $O_{ij}$, ($j = 1, . . . , k_i$), is used to estimate the offset:

$$ d_{cos}(\mathbf{w}_i', \mathbf{f}_{ij}) = 1 - \dfrac{\mathbf{w}_i' \cdot \mathbf{f}_{ij}}{\|\mathbf{w}_i'\| \ \|\mathbf{f}_{ij}\|} $$

where || · || is the magnitude of the corresponding vector. In this use case, being present only one asset, $n$=1.

The derived weight vectors represent individual direction measurements as a result of decomposing per individual direction. The weight vectors are appended into a single ($3\times h$)-dimensional vector $\mathbf{w}_i'$  of rank 1 that is compared to the fingerprint $f_{1j}$ of the corresponding operating mode $O_{1j}$.

Once the offset are extracted, it is possible to evaluate to what extent they can be exploited for anomaly detection. For this reason in next code section, for each operating mode are computed the offset of the derived weights to the corresponding fingerprint. More specifically, a pivot table is created below showing the cosine distance in order to compare distances between measurements and their corresponding fingerprints (left) and other fingerprints (right).

In [ ]:
df_cosine = get_pivot_table(df_W_online, fingerprints, test_vibration_measurement_periods_meta_data)

print(f'Pivot table with distances to all fingerprints (0 - {len(df_cosine.columns) - 7}), corresponding rpm and torque values, and additional information on the anomaly condition:')
display(df_cosine.head())

As next step, it is evaluated whether the performance offset can be exploited for detecting anomalies. In the plot below is analzed the relation between offset distance and pitting level.

In [ ]:
ax = sns.boxplot(data=df_cosine, y='distance_to_own_cluster_center', x='pitting_level')
ax.set_ylabel('Distance to context sensitive fingerprint')
ax.set_title(f'Distance to own cluster center per pitting level');

In the boxplots above, a clear difference in terms of offset between healthy and faulty gearboxes can be observed. Indeed, healthy gearboxes, the one with pitting level = 0, have offset close to 0 wherease faulty gearbox, the one with pitting level above 0, have higher offset. This is expected, as the context-sensitive fingerprint is derived from healthy data without pitting and data with pitting is likely to have vibration patterns not present in the healthy data set.

The boxplot provides only a partial view of the ability of the methodology to identify anomalies. To fully appreciate the performance of the methodology, it is necessary to adequately detect faulty gearboxes as anomalies.
Whether a datapoint is labelled as anomalous depends on a predefined distance threshold.
If the predefined distance threshold is exceeded, an anomaly is raised.
To make our validation independent of the choice of this hyperparameter, we construct a ROC-curve by varying this distance threshold.

A ROC curve is a visualisation for evaluating the performance of the anomaly detection. It plots the *true positive rate* (TPR) against the *false positive rate (FPR)* at different distance thresholds. In this use case, the TPR represents the rate of measurements exposed to pitting that were actually detected as anomaly whereas the FPR is the rate of measurements that were not exposed to pitting and still detected as anomaly.
The higher is the area under the ROC-curve, the better the model is at detecting anomalies.

Below the ROC-curve is plotted.

In [ ]:
plot_ROC_curve(df_cosine)

In the figure above it can be observed that the anomaly detection generally performs well with an area under the curve (AUC) of $0.979$.
Operators aim for a high TPR while minimizing false alarms (keeping the FPR low). Therefore, we additionally tracked the TPR at a stable FPR of 0.1 (TPR@FPR=0.1, red dashed line), which represents the TPR when there are 10% false positives. In this scenario, the TPR would be 96%. An alternatve objective criterium is to keep the FPR as low as possible at a high TPR.
The plot above therefore also illustrates the FPR@TPR=0.9 (green dashed line).In that scenario, we check how many false alarms would be triggered if we want to guarantee that 90% of the gear pitting is detected. In this use case, FPR@TPR=0.9 is 0.04.

## 4. Conclusions

In this SK is illustrated a data-driven methodology for contextual performance profiling and anomaly detection. The SK focuses on how to set up methodology parameters and correctly interpret its results. The methodology, validated on a gearbox that is subject to pitting, explains how to extract the contexts from an asset and how to use them to profile its performance. Furthermore, it proves that the performance profiles can be used to identify anomalies. For the latter, it has been shown that our method is able to detect most anomalies, while throwing few false alarms, as demonstrated through the ROC curve.

# Additional information
The methodology presented in this notebook is based on the papers [[1]](#fingerhut2023) and [[2]](#fingerhut2024).

This Starter Kit was developed in the context of the Compact project [ToDo: Find an online reference]. For more information, please contact [info@elucidata.be](mailto:info@elucidata.be).

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Notebook"), to deal in the Notebook without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Notebook, and to permit persons to whom the Notebook is provided to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies of the Notebook and/or copies of substantial portions of the Notebook.

THE NOTEBOOK IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NON-INFRINGEMENT. IN NO EVENT SHALL SIRRIS, THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, DIRECT OR INDIRECT, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE NOTEBOOK OR THE USE OR OTHER DEALINGS IN THE NOTEBOOK

# References

<a id='fingerhut2023'>[1]</a> F. Fingerhut, S. Klein, M. Verbeke, S. Rajendran and E. Tsiporkova, "Multi-view contextual performance profiling in rotating machinery," 2023 IEEE International Conference on Prognostics and Health Management (ICPHM), Montreal, QC, Canada, 2023, pp. 343-350, [doi: 10.1109/ICPHM57936.2023.10194172](https://ieeexplore.ieee.org/document/10194172).

<a id='fingerhut2024'>[2]</a> F. Fingerhut, M. Verbeke and E. Tsiporkova, "Unsupervised context-sensitive anomaly detection on streaming data relying on multi-view profiling," 2024 IEEE International Conference on Evolving and Adaptive Intelligent Systems (EAIS), Madrid, Spain, 2024, pp. 1-10, [doi: 10.1109/EAIS58494.2024.10569106](https://ieeexplore.ieee.org/document/10569106).

<a id='nmf'>[3]</a> D. Lee, S. Seung, [“Learning the parts of objects by non-negative matrix factorization”](https://www.cs.columbia.edu/~blei/fogm/2020F/readings/LeeSeung1999.pdf), 1999

#### ©, 2024, Sirris